In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn.model_selection as mod
import sklearn.neighbors as nei
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import LeaveOneOut
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score, classification_report, precision_recall_curve, average_precision_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFECV
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import operator
%matplotlib inline

In [16]:
#load data
pima = pd.read_csv('/content/pima-indians-diabetes.csv')

In [17]:
# Hiển thị tên cột
pima_col_idx = pima.columns
pima_col_idx

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

#Phân loại trạng thái dinh dưỡng dựa trên BMI

In [18]:
Nutritional_status = [None] * len(pima)

for i in range(len(pima)):
    if pima['BMI'][i] == 0.0:
        Nutritional_status[i] = "Không có dữ liệu"
    elif pima['BMI'][i] < 18.5:
        Nutritional_status[i] = "Thiếu cân"
    elif pima['BMI'][i] < 25:
        Nutritional_status[i] = "Bình thường"
    elif 25 <= pima['BMI'][i] < 30:
        Nutritional_status[i] = "Thừa cân"
    elif pima['BMI'][i] >= 30:
        Nutritional_status[i] = "Béo phì"
    else:
        Nutritional_status[i] = pima['BMI'][i]

In [19]:
# Insert new column
pima.insert(6, "Nutritional Status", Nutritional_status)
pima.head (5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,Nutritional Status,DiabetesPedigreeFunction,Age,Outcome
0,0,148,72,35,0,33.6,Béo phì,0.627,50,1
1,1,85,66,29,0,26.6,Thừa cân,0.351,31,0
2,8,183,64,0,0,23.3,Bình thường,0.672,32,1
3,1,89,66,23,94,28.1,Thừa cân,0.167,21,0
4,0,137,40,35,168,43.1,Béo phì,2.288,33,1


In [29]:
# số lượng trạng thái dinh dưỡng
vc = pima['Nutritional Status'].value_counts()

# Tính phần trăm (dựa trên tổng số mẫu)
phần_trăm = (vc / len(pima) * 100).round(2)

# Tạo DataFrame kết hợp số lượng và phần trăm
bang_ket_qua = pd.DataFrame({
    'Số lượng': vc,
    'Phần trăm (%)': phần_trăm
})
print(bang_ket_qua)


                    Số lượng  Phần trăm (%)
Nutritional Status                         
Béo phì                  472          61.46
Thừa cân                 179          23.31
Bình thường              102          13.28
Không có dữ liệu          11           1.43
Thiếu cân                  4           0.52


Dánh giá nhóm "Béo phì" chiếm tỷ lệ áp đảo (61.46%), trong khi "Thiếu cân" chỉ chiếm 0.52%, cho thấy dân số Pima có xu hướng béo phì cao.

#Phân loại dựa trên nồng độ glucose huyết tương sau 2 giờ

In [24]:
OGTT_Interpretation = pd.Series([])

for i in range(len(pima)):
    if pima['Glucose'][i] == 0.0:
        OGTT_Interpretation [i]="NA"

    elif pima['Glucose'][i] <= 140:
        OGTT_Interpretation [i]="Bình thường"

    elif pima['Glucose'][i] > 140 & pima['Glucose'][i] <= 198:
        OGTT_Interpretation [i]="Rối loạn dung nạp glucose"

    elif pima['Glucose'][i] > 198:
        OGTT_Interpretation[i]="Cấp độ tiểu đường"

    else:
        OGTT_Interpretation [i]= pima['Glucose'][i]

In [25]:
pima.insert(2, "Glucose Result", OGTT_Interpretation)
pima['Glucose Result'].value_counts()

,count
Glucose Result,
Bình thường,571
Rối loạn dung nạp glucose,192
NA,5


Đánh giá Không có cá thể nào trong mẫu cho kết quả ở mức "Tiểu đường"

In [32]:
Impaired_Glucose_Tolerance_Diabetic = ((pima ['Glucose'] > 140 ) & (pima ['Glucose'] <= 198) & (pima ['Outcome'] == 1)).sum()
Impaired_Glucose_Tolerance_Diabetic

np.int64(131)

có 131 bệnh nhân tiểu đường có mức glucose từ 140-198 mg/dL.

In [31]:
Normal_Glucose_Diabetic = ((pima ['Glucose'] != 0 ) & (pima ['Glucose'] <= 140) & (pima ['Outcome'] == 1)).sum()
Normal_Glucose_Diabetic

np.int64(134)

Có 134 bệnh nhân mắc bệnh tiểu đường có mức glucose ≤ 140 mg/dL.

#Đánh giá theo phân loại độ dày nếp gấp da

In [37]:
Percentile_skin_thickness = pd.Series([])

In [36]:
# Thống kê số lượng theo độ tuổi

pima['Age'].value_counts()

,count
Age,
22,72
21,63
25,48
24,46
23,38
28,35
26,33
27,32
29,29


In [38]:
# Khởi tạo Series trống cho Percentile_skin_thickness
Percentile_skin_thickness = pd.Series([])

for i in range(len(pima)):
    if pima["Age"][i] >= 20.0 and pima["Age"][i] <= 79.0:
        if pima["SkinThickness"][i] == 0.0:
            Percentile_skin_thickness[i] = "0 NA"
        elif pima["SkinThickness"][i] < 11.9:
            Percentile_skin_thickness[i] = "1 <P5th"
        elif pima["SkinThickness"][i] == 11.9:
            Percentile_skin_thickness[i] = "2 P5th"
        elif pima["SkinThickness"][i] > 11.9 and pima["SkinThickness"][i] < 14.0:
            Percentile_skin_thickness[i] = "3 P5th - P10th"
        elif pima["SkinThickness"][i] == 14.0:
            Percentile_skin_thickness[i] = "4 P10th"
        elif pima["SkinThickness"][i] > 14.0 and pima["SkinThickness"][i] < 15.8:
            Percentile_skin_thickness[i] = "5 P10th - P15th"
        elif pima["SkinThickness"][i] == 15.8:
            Percentile_skin_thickness[i] = "6 P15th"
        elif pima["SkinThickness"][i] > 15.8 and pima["SkinThickness"][i] < 18.0:
            Percentile_skin_thickness[i] = "7 P15th - P25th"
        elif pima["SkinThickness"][i] == 18.0:
            Percentile_skin_thickness[i] = "8 P25th"
        elif pima["SkinThickness"][i] > 18.0 and pima["SkinThickness"][i] < 23.5:
            Percentile_skin_thickness[i] = "9 P25th - P50th"
        elif pima["SkinThickness"][i] == 23.5:
            Percentile_skin_thickness[i] = "10 P50th"
        elif pima["SkinThickness"][i] > 23.5 and pima["SkinThickness"][i] < 29.0:
            Percentile_skin_thickness[i] = "11 P50th - P75th"
        elif pima["SkinThickness"][i] == 29.0:
            Percentile_skin_thickness[i] = "12 P75th"
        elif pima["SkinThickness"][i] > 29.0 and pima["SkinThickness"][i] < 31.9:
            Percentile_skin_thickness[i] = "13 P75th - P85th"
        elif pima["SkinThickness"][i] == 31.9:
            Percentile_skin_thickness[i] = "14 P85th"
        elif pima["SkinThickness"][i] > 31.9 and pima["SkinThickness"][i] < 33.7:
            Percentile_skin_thickness[i] = "15 P85th - P90th"
        elif pima["SkinThickness"][i] == 33.7:
            Percentile_skin_thickness[i] = "16 P90th"
        elif pima["SkinThickness"][i] > 33.7 and pima["SkinThickness"][i] < 35.9:
            Percentile_skin_thickness[i] = "17 P90th - P95th"
        elif pima["SkinThickness"][i] == 35.9:
            Percentile_skin_thickness[i] = "18 P95th"
        elif pima["SkinThickness"][i] > 35.9:
            Percentile_skin_thickness[i] = "19 >P95th"
    elif pima["Age"][i] >= 80.0:  # Only 1 woman is 81 years old
        if pima["SkinThickness"][i] > 31.7:
            Percentile_skin_thickness[i] = "20 >P95th"

In [39]:
pima.insert(4, "Percentile skin thickness", Percentile_skin_thickness)

In [40]:
pima.head(5)

,Pregnancies,Glucose,Glucose Result,BloodPressure,Percentile skin thickness,SkinThickness,Insulin,BMI,Nutritional Status,DiabetesPedigreeFunction,Age,Outcome
0,0,148,Rối loạn dung nạp glucose,72,17 P90th - P95th,35,0,33.6,Béo phì,0.627,50,1
1,1,85,Bình thường,66,12 P75th,29,0,26.6,Thừa cân,0.351,31,0
2,8,183,Rối loạn dung nạp glucose,64,0 NA,0,0,23.3,Bình thường,0.672,32,1
3,1,89,Bình thường,66,9 P25th - P50th,23,94,28.1,Thừa cân,0.167,21,0
4,0,137,Bình thường,40,17 P90th - P95th,35,168,43.1,Béo phì,2.288,33,1


In [41]:
pima['Percentile skin thickness'].value_counts()

,count
Percentile skin thickness,
0 NA,227
19 >P95th,145
11 P50th - P75th,87
9 P25th - P50th,79
15 P85th - P90th,50
13 P75th - P85th,46
17 P90th - P95th,23
7 P15th - P25th,20
8 P25th,20


Tổng số mẫu: 768 phụ nữ. Số lượng thiếu dữ liệu: 227 phụ nữ (khoảng 29.6%) không có thông tin về độ dày nếp gấp da (SkinThickness = 0), chiếm tỷ lệ đáng kể.Hầu hết các mẫu có phân vị độ dày da lớn hơn P95th (nhóm "19 >P95th" hoặc "20 >P95th"). 53 phụ nữ (khoảng 6.9%) có giá trị dưới P15th (nhóm "1 <P5th" đến "6 P15th"), có thể chỉ ra tình trạng suy dinh dưỡng. Chỉ 1 phụ nữ (0.13%) từ 80 tuổi trở lên, với phân vị > P95th.

In [42]:
diabetic_malnourished_st = ((pima ['SkinThickness'] < 15.8) & (pima ['Outcome'] == 1)).sum()
diabetic_malnourished_st

np.int64(94)

có 94 bệnh nhân tiểu đường có dấu hiệu suy dinh dưỡng dựa trên độ dày da.

In [ ]:
diabetic_malnourished_bmi = ((pima ['BMI'] < 18.5) & (pima ['Outcome'] == 1)).sum()
diabetic_malnourished_bmi

Có 2 bệnh nhân được chẩn đoán tiểu đường nhưng đồng thời bị suy dinh dưỡng theo chỉ số BMI

In [43]:
diabetic_malnourished_bmi_st = ((pima ['BMI'] < 18.5) & (pima ['SkinThickness'] < 15.8) & (pima ['Outcome'] == 1)).sum()
diabetic_malnourished_bmi_st

np.int64(2)

Có 2 bệnh nhân tiểu đường có cả BMI < 18.5 và độ dày da < 15.8 mm.

Quan sát thú vị khi ghi nhận rằng 94 phụ nữ mắc tiểu đường được phân loại là thiếu cân/suy dinh dưỡng dựa chỉ trên độ dày nếp gấp da (SkinThickness), nhưng chỉ có 2 phụ nữ mắc tiểu đường được xác định là thiếu cân/suy dinh dưỡng khi xem xét chỉ số BMI hoặc kết hợp BMI và độ dày nếp gấp da. Việc đo độ dày nếp gấp da là một phép đo khó thực hiện chính xác và đáng tin cậy nếu không có đào tạo nghiêm ngặt, vì lý do đó, phương pháp này hiếm được sử dụng ngày nay và không áp dụng phổ quát.